<a href="https://colab.research.google.com/github/GijsBarmentlo/TweetNLP/blob/master/NLP_Disaster_tweet_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports + open drive + Kaggle API
Setup environment and link to google drive for file storage and Kaggle API for submit

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
#setup kaggle for submission

!pip install kaggle
!mkdir .kaggle
!ls -a

mkdir: cannot create directory ‘.kaggle’: File exists
.  ..  .config	gdrive	.kaggle  sample_data


In [0]:
import json
token = {"username": "gijske" ,"key":"9e096df0b412d13bc210ea18ad893557"}

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!chmod 600 /content/.kaggle/kaggle.json

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json

In [6]:
!kaggle config set -n path -v{/content}

- path is now set to: {/content}


In [7]:
#use my drive as HDD
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
colab_dir = "/content/gdrive/My Drive/Colab Data/"
root_dir = "/content/gdrive/My Drive/Colab Data/Tweet_classification/"

Mounted at /content/gdrive


# Data preparation

## reading data
Read train and test data (.csv) as dataframes

In [0]:
#imports
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split

In [0]:
train_df = pd.read_csv(root_dir + "train.csv")
test_df = pd.read_csv(root_dir + "test.csv")

In [10]:
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


## Keyword : Cleaning and handling NaN


In [11]:
train_df.keyword.nunique()

221

In [12]:
train_df.keyword.unique()

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

there are 221 unique values, spaces are saved as "%20", we will need to clean that in preprocessing

In [0]:
def clean_keyword(keyword):
  if isinstance(keyword, str):
    return keyword.replace('%20',' ')
  else :
    return ""

In [14]:
train_df.keyword.isna().sum()

61

missing values will need to be handled

In [15]:
test_df.keyword.nunique()

221

In [16]:
test_df.keyword.unique()

array([nan, 'ablaze', 'accident', 'aftershock', 'airplane%20accident',
       'ambulance', 'annihilated', 'annihilation', 'apocalypse',
       'armageddon', 'army', 'arson', 'arsonist', 'attack', 'attacked',
       'avalanche', 'battle', 'bioterror', 'bioterrorism', 'blaze',
       'blazing', 'bleeding', 'blew%20up', 'blight', 'blizzard', 'blood',
       'bloody', 'blown%20up', 'body%20bag', 'body%20bagging',
       'body%20bags', 'bomb', 'bombed', 'bombing', 'bridge%20collapse',
       'buildings%20burning', 'buildings%20on%20fire', 'burned',
       'burning', 'burning%20buildings', 'bush%20fires', 'casualties',
       'casualty', 'catastrophe', 'catastrophic', 'chemical%20emergency',
       'cliff%20fall', 'collapse', 'collapsed', 'collide', 'collided',
       'collision', 'crash', 'crashed', 'crush', 'crushed', 'curfew',
       'cyclone', 'damage', 'danger', 'dead', 'death', 'deaths', 'debris',
       'deluge', 'deluged', 'demolish', 'demolished', 'demolition',
       'derail', 'der

In [17]:
test_df.keyword.unique() == train_df.keyword.unique()

array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

Keywords are the same in both test and train data, which means we can oh encode them if we want. This could pose a problem for generalisation

## Cleaning and lemming text
Using spacy, remove stopwords, hashtags, mentions and punctuation.
Function definition only, no modifications made to data

In [0]:
import spacy
nlp = spacy.load('en')

def lemmatize_text(input_string, remove_stopwords = True):
  lemmatized_text = ""

  spacy_doc = nlp(input_string)
  if remove_stopwords == False:
    for token in spacy_doc:
      lemmatized_text = lemmatized_text + ' ' + token.lemma_

  if remove_stopwords:
    for token in spacy_doc:
      if token.is_stop == False:
        lemmatized_text = lemmatized_text + " " + token.lemma_

  return lemmatized_text

def remove_mentions_hashtags(input_text):
  """
  Removes # and @
  """
  clean_text = re.sub(r'@\w+', '', input_text)
  clean_text = clean_text.replace("#", "")
  return clean_text


def remove_punctuation(input_text):
  """
  Takes input string and returns string without punctuation
  """
  clean_text = re.sub(r'[^\w\s]','',input_text)
  return clean_text

In [0]:
#lemmatize_text("we are doing this better, i think", remove_stopwords = False)

## Feature generation
Using NLTK, these functions generate features from the input text. Function definition only, no modifications made to data

In [20]:
from nltk.tokenize import RegexpTokenizer, TweetTokenizer, sent_tokenize
import re
import nltk
nltk.download("popular")
from nltk import *
from nltk.corpus import stopwords

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

In [0]:
def sentence_count(input_text):
    """
    count the number of sentences in tweet
    """
    count = len(sent_tokenize(input_text))
    return count

def word_count(input_text):
    '''
    Counts words in input string
    '''
    re_tokenizer = RegexpTokenizer(r'\w+') #using regexp to not count punctuation
    tokens = re_tokenizer.tokenize(input_text)

    word_count = 0
    for token in tokens:
      word_count += 1
            
    return word_count


def stopword_ratio(input_text):
    """
    return ration of stopwords on total words in tweet
    """
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(input_text)
    word_count = len(tokens)
    
    StopWords = set(stopwords.words('english'))
    stopword_count = 0
    for token in tokens:
        if token in StopWords:
            stopword_count += 1
    
    ratio = 0
    if word_count != 0:
        ratio = stopword_count / word_count
    
    return ratio


def count_hashtags(input_text):
  """
  Counts the number of hashtags in the text
  """
  return input_text.count('#')


def count_mentions(input_text):
  """
  counts mentions in the text, N.B. mentions are @someone
  """
  return input_text.count('@')

def count_punctuation(input_tex):
  """
  counts all punctuation
  """
  return input_text.count(string.punctuation)

## Wrapper
Defines a function "text_preprocessing(df)" which applies both cleaning, lemming and feature generation to a dataframe's "text" column

In [0]:
def text_preprocessing(df):
  df['keyword'] = (df['keyword'].apply(clean_keyword))
  df['lemmatized_text'] = (df['text']).apply(remove_mentions_hashtags).apply(lemmatize_text)
  df['sentence_count'] = (df['text']).apply(sentence_count)
  df['mention_count'] = (df['text']).apply(count_mentions)
  df['hashtag_count'] = (df['text']).apply(count_hashtags)
  df['word_count'] = (df['text']).apply(remove_mentions_hashtags).apply(word_count)
  df['stopword_ratio'] = (df['text']).apply(remove_mentions_hashtags).apply(stopword_ratio)
  return df

generated_features = ['sentence_count', 'mention_count', 'hashtag_count', 'word_count',
       'stopword_ratio']
generated_features_count = len(generated_features)

# Word embedding

## Spacy pre-trained
Using spacy's pretrained 300dim word embedding, create a numpy array composed of the numerical features we generated from the text and the sum of each word's embedded vector

In [23]:
#import spacy
!python -m spacy download en_core_web_lg
!pip install -U spacy[cuda100]
gpu = spacy.prefer_gpu()
import en_core_web_lg
nlp = en_core_web_lg.load()

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
Requirement already up-to-date: spacy[cuda100] in /usr/local/lib/python3.6/dist-packages (2.2.4)


In [0]:
def make_X(df, chosen_features = generated_features, column_to_emb = 'lemmatized_text'):

  df = text_preprocessing(df)

  if column_to_emb not in df.columns :
    printf(f'dataframe does not contain column named {column_to_emb}')
    return

  with nlp.disable_pipes():
    doc_vecs = np.array([nlp(text).vector for text in df['text']])
    keyword_vec = np.array([nlp(keyword).vector for keyword in df['keyword']])
    X = np.hstack((doc_vecs, df[chosen_features]))
    X = np.hstack((X,keyword_vec))

  return X

In [0]:
#Todo make a clean embedder funciton that just takes text column and returns np.array ; WIP

def spacy_embed(df, column_to_emb = 'lemmatized_text'):

  df = text_preprocessing(df)

  if column_to_emb not in df.columns :
    printf(f'dataframe does not contain column named {column_to_emb}')
    return

  with nlp.disable_pipes():
    emb_vecs = [nlp(text).vector for text in df[column_to_emb]]
    X = np.array(emb_vecs)

  return X

#embedded_text = spacy_embed(df, column_to_emb = 'lemmatized_text')

## ToDo : My own implementation of word2vec

# Models
We're gonna try a few types of models, simple first, complicated later. This will allow us to benchmark performance and avoid needless effort.

## Ridge classifier
They perform well on this type of tasks as they are better at taking weak signals into account

In [0]:
#basic ridge classifier, used for handling n>m & adding needed bias
ridge_clf = linear_model.RidgeClassifierCV()

#basic evaluation
scores = model_selection.cross_val_score(ridge_clf, X, train_df.target, cv=10, scoring="f1")
print(f"average f1 score with CV {scores.mean()}")
scores = model_selection.cross_val_score(ridge_clf, X, train_df.target, cv=10, scoring="accuracy")
print(f"average accuracy score with CV {scores.mean()}")

#fit model
ridge_clf.fit(X,train_df.target)

average f1 score with CV 0.7174758499213684
average accuracy score with CV 0.770393114461218


RidgeClassifierCV(alphas=array([ 0.1,  1. , 10. ]), class_weight=None, cv=None,
                  fit_intercept=True, normalize=False, scoring=None,
                  store_cv_values=False)

We get 0.72 f1 and 76% accuracy, let's improve with a simple neural network

## Random forest
Same reasoning here, a rf will make sure some trees are optimised for the weaker features which seems key in an NLP task

In [0]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier()

scores = model_selection.cross_val_score(rf_clf, X, train_df.target, cv=7, scoring="f1")
print(f"average f1 score with CV {scores.mean()}")
scores = model_selection.cross_val_score(rf_clf, X, train_df.target, cv=7, scoring="accuracy")
print(f"average accuracy score with CV {scores.mean()}")

average f1 score with CV 0.6922235464996395
average accuracy score with CV 0.7627778974975455


## Dense NN
Let's see what results we can get with a dense neural network, sticking to Keras for simplicity and legibility

### Loss function
We will evaluate this model by F1 score as it is a binary classification problem and there is no reason to give big priority to either precision or recall. There is no longer a built F1 score in Keras so we will need to implement a differentiable F1 ourselves


In [26]:
#F1 score and differentiable f1 loss

import keras.backend as K

def f1_score(y_true, y_pred): #taken from old keras source code
  true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
  possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
  predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
  precision = true_positives / (predicted_positives + K.epsilon())
  recall = true_positives / (possible_positives + K.epsilon())
  f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
  return f1_val


# Taken from this kernel https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric , my thanks to Michal
import tensorflow as tf

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

Using TensorFlow backend.


### Helper functions
A plot function that will take history as input and plot loss, accuracy and f1, this will make it easier to quickly evaluate performance and iterate over hyperparameters for the the NN 

In [0]:
#graph results

import matplotlib.pyplot as plt 

def plot_history(history):
  keys = history.history.keys()
  metric_present = False

  #Loss graph
  for key in keys :
    if 'loss' in key:
      plt.plot(history.history[key])
      metric_present = True

  if metric_present:
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['val', 'train'], loc='upper left')
    plt.show()
    metric_present = False


  #accuracy graph
  for key in keys :
    if 'acc' in key:
      plt.plot(history.history[key])
      metric_present = True
    
  if metric_present:
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['val', 'train'], loc='upper left')
    plt.show()
    metric_present = False

  #f1 graph
  for key in keys :
    if 'f1' in key:
      plt.plot(history.history[key])
      metric_present = True
    
  if metric_present:
    plt.title('F1 score')
    plt.ylabel('F1')
    plt.xlabel('epoch')
    plt.legend(['val', 'train'], loc='upper left')
    plt.show()
    metric_present = False  

  return

### Prepare datasets

In [28]:
# Make a numpy array as input for the NN, 
X_nf = make_X(train_df, column_to_emb = 'lemmatized_text')
X_train_nf , X_val_nf, y_train_nf, y_val_nf = train_test_split(X_nf, train_df.target , test_size=0.2, random_state=42)
X_nf.shape

(7613, 605)

### Model

In [0]:
# Packages for modeling
from keras import models
from keras import layers
from keras import regularizers

In [0]:
def make_model(input_dimension = X_nf.shape[1], layer_list = [600, 300, 100, 100, 100, 100, 100, 100, 100, 100, 100], optimiser = 'adam'):
  
  #Make model and input layer
  made_model = models.Sequential()
  made_model.add(layers.Dense(600, input_dim = input_dimension, activation = 'relu'))

  #Add hidden layers
  for number_of_neurons in layer_list:
    made_model.add(layers.Dense(number_of_neurons, activation = 'relu'))

  #Add output layer
  made_model.add(layers.Dense(1, activation = 'sigmoid'))

  #Compile
  made_model.compile(loss = f1_loss, optimizer = optimiser, metrics=['accuracy', f1_score])

  #print summary
  printf('made this model with made_model function')
  printf(made_model.summary())

  return made_model

In [0]:
thin_model = make_model(layer_list = [600, 300, 100, 100, 100, 100, 100, 100])
thin_model.summary()
thin_history = thin_model.fit(X_train_nf, y_train_nf, batch_size = 1015, epochs = 150, validation_data = (X_val_nf, y_val_nf))

plot_history(thin_history)

Train on 6090 samples, validate on 1523 samples
Epoch 1/150
6090/6090 [==============================] - 108s 18ms/step - loss: 0.4580 - accuracy: 0.4371 - f1_score: 0.5948 - val_loss: 0.4019 - val_accuracy: 0.4261 - val_f1_score: 0.5974
Epoch 2/150
6090/6090 [==============================] - 107s 18ms/step - loss: 0.3979 - accuracy: 0.4305 - f1_score: 0.6019 - val_loss: 0.4024 - val_accuracy: 0.4261 - val_f1_score: 0.5974
Epoch 3/150
6090/6090 [==============================] - 107s 18ms/step - loss: 0.3983 - accuracy: 0.4305 - f1_score: 0.6017 - val_loss: 0.4024 - val_accuracy: 0.4261 - val_f1_score: 0.5974
Epoch 4/150
6090/6090 [==============================] - 107s 18ms/step - loss: 0.3981 - accuracy: 0.4305 - f1_score: 0.6019 - val_loss: 0.4024 - val_accuracy: 0.4261 - val_f1_score: 0.5974
Epoch 5/150
6090/6090 [==============================] - 107s 18ms/step - loss: 0.3982 - accuracy: 0.4305 - f1_score: 0.6018 - val_loss: 0.4024 - val_accuracy: 0.4261 - val_f1_score: 0.5974
Ep

In [0]:
#Need to use save weights because save_model doesn't work with custom loss functions
#nf_model.save_weights("nf_model_deep_bigBatch_keyword.h5")

### Make prediction & submit

In [32]:
sub_model = thin_model
prediction_np = sub_model.predict_classes(make_X(test_df))
#prediction_np = (prediction_np > 0.5 )
prediction_df = pd.DataFrame(prediction_np,columns = ['target'])
prediction_df['id'] = test_df.id
#prediction_df.target = prediction_df.target.apply(lambda x : 1 if x==True else 0)
prediction_df.head()

,target,id
0,1,0
1,1,2
2,1,3
3,1,9
4,1,11


In [0]:
prediction_df.to_csv('submission.csv', index = False)

In [34]:
!kaggle competitions submit -c nlp-getting-started -f submission.csv -m "neural_network, features = lemma_text_emb + keyword_emb + gen_features, batches of 1000"

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.
